# IHT Experiments to verify theory for L-0 Norm - CIFAR-10

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division
from __future__ import print_function

import sys, os, gc, math
import numpy as np
from scipy.fftpack import dct,idct
from keras.datasets import mnist, fashion_mnist,cifar10
from PIL import Image


sys.path.append('../')

from models.util import *


#Seed used for choosing classes, training points, and test points.
#SEED = 14
SEED=11

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
num_samples = 500
sqrt_n = 32
input_shape=(sqrt_n,sqrt_n,1)
n = sqrt_n*sqrt_n
k = 5
c=2.0

In [3]:
#Load Cifar10 data
(X_train, _), (X_test, _) = cifar10.load_data()
m_data = np.concatenate((X_train,X_test))

In [4]:
#Normalize the data
m_data = m_data/255.0

In [5]:
#Check Cifar10 results for 500 random images - BP
t = int(float(n)/27.0/k/c)
print('Noise budget of %d' % t)
subset_idx = np.random.choice(np.arange(m_data.shape[0]),num_samples)
m_data_sub_bp = m_data[subset_idx]
m_data_y_bp = np.zeros((num_samples,sqrt_n,sqrt_n,3))
t_values_bp = np.zeros((num_samples,3),dtype=np.int8)
for i in range(num_samples):
    #first sample an element from the data
    x_r = m_data_sub_bp[i,:,:,0].flatten()
    x_g = m_data_sub_bp[i,:,:,1].flatten()
    x_b = m_data_sub_bp[i,:,:,2].flatten()

    #Now sample a t - must be atleast 1
    t_r = np.random.randint(1,t)
    t_g = np.random.randint(1,t)
    t_b = np.random.randint(1,t)
    t_values_bp[i,0] = t_r
    t_values_bp[i,1] = t_g
    t_values_bp[i,2] = t_b


    #Now samnple the an index set
    s_r = np.random.choice(np.arange(n),t_r)
    s_g = np.random.choice(np.arange(n),t_g)
    s_b = np.random.choice(np.arange(n),t_b)


    e = np.zeros((n,3))
    #Now create the vector e
    #pick a value for each element between 0 and 1 as the images are normalized
    for j in range(t_r):
        e[s_r[j],0] = np.random.uniform()
    for j in range(t_g):
        e[s_g[j],1] = np.random.uniform()
    for j in range(t_b):
        e[s_b[j],2] = np.random.uniform()
    y = m_data_sub_bp[i,:,:,:] + e.reshape((sqrt_n,sqrt_n,3))
    m_data_y_bp[i,:,:,:] = y

Noise budget of 3


# Get \delta_p and \Delta_p

In [6]:
%%capture three
errors_l2 = np.zeros(m_data_y_bp.shape[0])
errors_inf = np.zeros(m_data_y_bp.shape[0])
bot_l2 = np.zeros((m_data_y_bp.shape[0],3))
diff_l2 = np.zeros(m_data_y_bp.shape[0])
diff_inf = np.zeros(m_data_y_bp.shape[0])
T_values = np.zeros((m_data_y_bp.shape[0],3))
tau = np.zeros((m_data_y_bp.shape[0],3))

for i in range(num_samples):
    y_r = m_data_y_bp[i,:,:,0].flatten()
    x_r = m_data_sub_bp[i,:,:,0].flatten()
    
    #Get actual top k and bottom k (we use the faster transform which may introduce some error)
    x_hat_top_k_r, x_hat_bot_k_r =  get_top_bot_k_vec(dct(x_r, norm='ortho'),k=k)    
    e_r = y_r - x_r 
    eta_r = bot_l2[i,0] = np.linalg.norm(x_hat_bot_k_r)
    
    a_r = np.sqrt(np.linalg.norm(x_hat_top_k_r)**2 + np.linalg.norm(eta_r)**2)
    rho_r = np.sqrt(27)*np.sqrt(c*k*t_values_bp[i,0]/float(n))
    
    #We want to find a T such that rho^T a is very small - we use 1e-20 
    T_r = int((np.log(1e-20) - np.log(a_r))/np.log(rho_r))
    T_values[i,0] = T_r
    
    y_g = m_data_y_bp[i,:,:,1].flatten()
    x_g = m_data_sub_bp[i,:,:,1].flatten()
    
    #Get actual top k and bottom k (we use the faster transform which may introduce some error)
    x_hat_top_k_g, x_hat_bot_k_g =  get_top_bot_k_vec(dct(x_g, norm='ortho'),k=k)    
    e_g = y_g - x_g
    eta_g = bot_l2[i,1] = np.linalg.norm(x_hat_bot_k_g)
    
    a_g = np.sqrt(np.linalg.norm(x_hat_top_k_g)**2 + np.linalg.norm(eta_g)**2)
    rho_g = np.sqrt(27)*np.sqrt(c*k*t_values_bp[i,1]/float(n))
    
    #We want to find a T such that rho^T a is very small - we use 1e-20 
    T_g = int((np.log(1e-20) - np.log(a_g))/np.log(rho_g))
    T_values[i,1] = T_g
    
    y_b = m_data_y_bp[i,:,:,2].flatten()
    x_b= m_data_sub_bp[i,:,:,2].flatten()
    
    #Get actual top k and bottom k (we use the faster transform which may introduce some error)
    x_hat_top_k_b, x_hat_bot_k_b =  get_top_bot_k_vec(dct(x_b, norm='ortho'),k=k)    
    e_b = y_b- x_b
    eta_b = bot_l2[i,2] = np.linalg.norm(x_hat_bot_k_b)
    
    a_b = np.sqrt(np.linalg.norm(x_hat_top_k_b)**2 + np.linalg.norm(eta_b)**2)
    rho_b = np.sqrt(27)*np.sqrt(c*k*t_values_bp[i,2]/float(n))
    
    #We want to find a T such that rho^T a is very small - we use 1e-20 
    T_b = int((np.log(1e-20) - np.log(a_b))/np.log(rho_b))
    T_values[i,1] = T_b
    
 
    x_hat_approx_r,_ = iht(y_r,t_values_bp[i,0],T=T_r,k=k)
    x_hat_approx_g,_ = iht(y_g,t_values_bp[i,1],T=T_g,k=k)
    x_hat_approx_b,_ = iht(y_b,t_values_bp[i,2],T=T_b,k=k)
    
    x_hat_approx_top_k_r = get_topk_vec(x_hat_approx_r,k=k)
    x_hat_approx_top_k_g = get_topk_vec(x_hat_approx_g,k=k)
    x_hat_approx_top_k_b = get_topk_vec(x_hat_approx_b,k=k)
    
    #Note the norm of bottom k coefficients
    bot_l2[i,0] = np.linalg.norm(x_hat_bot_k_r)
    bot_l2[i,1] = np.linalg.norm(x_hat_bot_k_g)
    bot_l2[i,2] = np.linalg.norm(x_hat_bot_k_b)



    #Get the multiplicative constant
    c_l2_r = np.sqrt((4*c*k*t_values_bp[i,0])/float(n))
    c_inf_r = np.sqrt( (2*c*t_values_bp[i,0])/float(n))
    
    c_l2_g = np.sqrt((4*c*k*t_values_bp[i,1])/float(n))
    c_inf_g = np.sqrt( (2*c*t_values_bp[i,1])/float(n))
    
    c_l2_b = np.sqrt((4*c*k*t_values_bp[i,2])/float(n))
    c_inf_b = np.sqrt( (2*c*t_values_bp[i,2])/float(n))
    
    rho_r = np.sqrt(27)*np.sqrt((c*k*t_values_bp[i,0])/float(n))
    rho_g = np.sqrt(27)*np.sqrt((c*k*t_values_bp[i,1])/float(n))
    rho_b = np.sqrt(27)*np.sqrt((c*k*t_values_bp[i,2])/float(n))

    #Calculate tau
    tau[i,0] = (np.sqrt(3)*np.sqrt(1 + 2*np.sqrt((c*k*t_values_bp[i,0])/float(n))))/(1-rho_r)
    tau[i,1] = (np.sqrt(3)*np.sqrt(1 + 2*np.sqrt((c*k*t_values_bp[i,1])/float(n))))/(1-rho_r)
    tau[i,2] = (np.sqrt(3)*np.sqrt(1 + 2*np.sqrt((c*k*t_values_bp[i,2])/float(n))))/(1-rho_r)

    #Note the errors
    errors_l2[i] = np.linalg.norm(x_hat_top_k_r.flatten()- x_hat_approx_r.flatten()) + \
                        np.linalg.norm(x_hat_top_k_g.flatten()- x_hat_approx_g.flatten()) + \
                         np.linalg.norm(x_hat_top_k_b.flatten()- x_hat_approx_b.flatten())
    #Note the errors
    errors_inf[i] = np.linalg.norm(x_hat_top_k_r.flatten()- x_hat_approx_r.flatten(),ord=np.inf) + \
                        np.linalg.norm(x_hat_top_k_g.flatten()- x_hat_approx_g.flatten(),ord=np.inf) + \
                         np.linalg.norm(x_hat_top_k_b.flatten()- x_hat_approx_b.flatten(),ord=np.inf)
    
    #Calculate the difference from the upper bound
    diff_l2[i] = (c_l2_r*tau[i,0]*bot_l2[i,0] + c_l2_g*tau[i,1]*bot_l2[i,1] + \
                    c_l2_b*tau[i,2]*bot_l2[i,2])- errors_l2[i] 
    diff_inf[i] = (c_inf_r*tau[i,0]*bot_l2[i,0] + c_inf_g*tau[i,1]*bot_l2[i,1] + \
                    c_inf_b*tau[i,2]*bot_l2[i,2])- errors_inf[i] 
    

In [7]:
print(np.mean(t_values_bp), 
      np.mean(errors_l2), 
      np.mean(diff_l2),
      np.mean(errors_inf),
      np.mean(diff_inf))

1.5026666666666666 0.27767200546120996 17.349919199824182 0.19278638476583795 5.3815474023396215


In [8]:
mnist_tup_bp = (m_data_y_bp, m_data_sub_bp, t_values_bp,errors_l2, diff_l2,errors_inf,diff_inf )

import pickle
with open('data/cifar10_theory_iht_l0.pickle', 'wb') as f:
    pickle.dump(mnist_tup_bp, f)
